In [7]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Dataset
import torchvision.datasets as datasets
from torchvision.datasets import CIFAR10
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter  

import warnings
warnings.filterwarnings("ignore")

In [3]:
### Hyperparameters

device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 3072
image_dim = 32 * 32 * 3  # 784
batch_size = 32
num_epochs = 50

In [4]:
# dataset module

class CIFARDataset(Dataset):
    '''
    downloads MNIST dataset, performs splitting and transformation, and returns dataloaders
    '''
    def __init__(self, root = './data', download = True, transform = None):
        # download mnist dataset
        self.cifar = CIFAR10(root = root, download = download)

        # default transformation if no specific transformation is provided
        if transform is None:
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))
            ])
        else:
            self.transform = transform

        self.indices = list(range(len(self.cifar)))

    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        img, _ = self.cifar[self.indices[idx]]
    
        if self.transform:
            img = self.transform(img)

        return img
    
    def get_dataloader(self, batch_size = batch_size, shuffle = True):
        return DataLoader(self, batch_size = batch_size, shuffle = shuffle)


NameError: name 'Dataset' is not defined

In [10]:
train_dataset = CIFARDataset()
train_dataloader = train_dataset.get_dataloader()


In [14]:
# discriminator class
class Discriminator(nn.Module):
    def __init__(self, in_channels = 3):
        super().__init__()
        # Simple CNN
        self.in_channels = in_channels

        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=4, stride = 2, padding = 1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride = 2, padding = 1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size = 4, stride = 2, padding = 1)

        self.bn1 = nn.BatchNorm2d(128)
        self.bn2 = nn.BatchNorm2d(256)

        self.fc = nn.Linear(256 * 4 * 4, 1)

  
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2, inplace = True)
        x = F.leaky_relu(self.bn1(self.conv2(x)), 0.2, inplace = True)
        x = F.leaky_relu(self.bn2(self.conv3(x)), 0.2, inplace = True)
        # Flatten the tensor so it can be fed into the FC layers
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return torch.sigmoid(x)


In [12]:
class Generator(nn.Module):
    '''
    Generates new images from random noise
    in: latent_dim 256*8*8
    out: 32x32
    '''
    def __init__(self, z_dim):
        super().__init__()
        self.gen = nn.Sequential(
        nn.Linear(z_dim, 8*8*64), # [32, 256] -> [32, 4096]
        nn.ReLU(),
        nn.Unflatten(1, (64, 8, 8)), # [32, 4096] -> [32, 64, 8, 8]
        nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding = 1),  # [32, 64, 8, 8] -> [32, 32, 16, 16]
        nn.ReLU(),
        nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding = 1),  # [32, 32, 16, 16] -> [32, 16, 32, 32]
        nn.ReLU(),
        )
        self.conv = nn.Conv2d(16, 3, kernel_size = 3, padding = 1) # [32, 16, 32, 32] -> [32, 3, 32, 32]
    
    def forward(self, x):
        return self.conv(self.gen(x))

In [ ]:
disc = Discriminator().to(device)
gen = Generator(z_dim).to(device)

fixed_noise = torch.randn((batch_size, z_dim)).to(device)

opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()

writer_fake = SummaryWriter(f"cifar_logs/fake")
writer_real = SummaryWriter(f"cifar_logs/real")
step = 0

for epoch in range(num_epochs):
    for batch_idx, (real) in enumerate(tqdm(train_dataloader)):
        # real is a batched tensor of the shape (batch_size, *img_dims) *img_dims = 1, 28, 28 for MNIST
        real = real.to(device)
        batch_size = real.shape[0]

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        ## The Discriminator must always be trained first to provide meaningful gradients for the generator
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        # torch.ones_like function creates a tensor of the same shape as the input tensor filled with ones
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        ## Here only the fake samples are passed to the discriminator as only those matter in measuring the quality of the generator
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad() # clears old gradients
        lossG.backward() # computes new gradients
        opt_gen.step() # updates weights based on gradients

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(train_dataloader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                data = real
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "CIFAR Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "CIFAR Real Images", img_grid_real, global_step=step
                )
                step += 1
    # Save the model checkpoints
    torch.save(disc.state_dict(), "cifar_disc.pth")
    torch.save(gen.state_dict(), "cifar_gen.pth")
    

  0%|          | 0/1563 [00:00<?, ?it/s]

Epoch [0/50] Batch 0/1563                       Loss D: 0.7483, loss G: 4.2223


  0%|          | 1/1563 [00:00<04:34,  5.70it/s]

Epoch [1/50] Batch 0/1563                       Loss D: 0.0091, loss G: 6.1035


  0%|          | 2/1563 [00:00<02:12, 11.74it/s]

...

  0%|          | 1/1563 [00:00<04:12,  6.20it/s]

Epoch [48/50] Batch 0/1563                       Loss D: 0.0073, loss G: 5.5699


  0%|          | 1/1563 [00:00<03:38,  7.14it/s]

Epoch [49/50] Batch 0/1563                       Loss D: 0.0032, loss G: 8.5442


100%|██████████| 1563/1563 [01:11<00:00, 21.89it/s]


In [2]:
# Generate Sample images from random noise
import torch

with torch.no_grad():
    noise = torch.randn(64, z_dim).to(device)
    fake_images = gen(noise)
    img_grid = torchvision.utils.make_grid(fake_images, normalize=True)

NameError: name 'z_dim' is not defined

In [ ]:
# training loop

In [ ]:
# inference block